In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'brain-tumor-mri-dataset' dataset.
Path to dataset files: /kaggle/input/brain-tumor-mri-dataset


In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# Configuração para garantir reprodutibilidade
np.random.seed(42)
tf.random.set_seed(42)

def carregar_dataset(base_dir, size=(64, 64)):
    X, y = [], []
    # Verifica se o diretório existe para evitar crash
    if not os.path.exists(base_dir):
        print(f"Diretório não encontrado: {base_dir}")
        return np.array([]), np.array([]), {}

    labels = sorted(os.listdir(base_dir))
    label_map = {label: i for i, label in enumerate(labels)}

    print("Lendo imagens...")
    for label in labels:
        folder = os.path.join(base_dir, label)
        for file in os.listdir(folder):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                try:
                    path = os.path.join(folder, file)
                    img = Image.open(path).convert('L') # Escala de cinza
                    img = img.resize(size)
                    X.append(np.array(img) / 255.0)
                    y.append(label_map[label])
                except Exception as e:
                    print(f"Erro no arquivo {file}: {e}")

    return np.array(X), np.array(y), label_map

# Uso (ajuste o caminho conforme sua pasta)
X_raw, y_raw, labels_map = carregar_dataset(f"{path}/Training")


# IMPORTANTE: Adiciona o canal de cor (1 para cinza)
X = X_raw.reshape(-1, 64, 64, 1)
y = y_raw

# Separa em treino e validação (essencial para saber se está aprendendo mesmo)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape X Treino:", X_train.shape)
print("Classes:", labels_map)

Shape X Treino: (80, 64, 64, 1)
Classes: {'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


In [ ]:
def criar_modelo_cnn():
    model = models.Sequential()

    # Substitui seu 'conv2d' e 'relu'
    # Usamos 32 filtros (kernels) diferentes, não apenas 1
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))

    # Substitui seu 'max_pooling'
    model.add(layers.MaxPooling2D((2, 2)))

    # Camada extra para aprender formas mais complexas (que seu código original não tinha)
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Prepara para a camada densa (Flat)
    model.add(layers.Flatten())

    # Camada densa (Hidden Layer)
    model.add(layers.Dense(64, activation='relu'))

    # Saída (Softmax) - 4 classes
    model.add(layers.Dense(4, activation='softmax'))

    return model

model = criar_modelo_cnn()
model.summary() # Mostra o tamanho das saídas (como seu print flat_size)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       802,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 821,956 (3.14 MB)

 Trainable params: 821,956 (3.14 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compila o modelo
# Optimizer 'adam': Ajusta a taxa de aprendizado (lr) automaticamente
# Loss 'sparse_categorical...': Igual à sua entropia cruzada para números inteiros
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# O batch_size=32 atualiza os pesos a cada 32 imagens (mais estável que 1 a 1)
print("\nIniciando treinamento...")
history = model.fit(X_train, y_train,
                    epochs=15,
                    batch_size=32,
                    validation_data=(X_val, y_val)) # Testa no conjunto de validação a cada época


Iniciando treinamento...
Epoch 1/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 244ms/step - accuracy: 0.1773 - loss: 1.5413 - val_accuracy: 0.2000 - val_loss: 1.4093
Epoch 2/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - accuracy: 0.2883 - loss: 1.3836 - val_accuracy: 0.1500 - val_loss: 1.3867
Epoch 3/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 242ms/step - accuracy: 0.2656 - loss: 1.3846 - val_accuracy: 0.2500 - val_loss: 1.3742
Epoch 4/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 242ms/step - accuracy: 0.2922 - loss: 1.3760 - val_accuracy: 0.4500 - val_loss: 1.3744
Epoch 5/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 594ms/step - accuracy: 0.3359 - loss: 1.3635 - val_accuracy: 0.4500 - val_loss: 1.3727
Epoch 6/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.3172 - loss: 1.3507 - val_accuracy: 0.4500 - val_loss: 1.3541
Epoch 7/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step - accuracy: 0.2867 - loss: 1.3326 - val_accuracy: 0.4500 - val_loss: 1.3499
Epoch 8/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step - accuracy: 0.3117 - loss: 1.3112 - val_ac

In [ ]:
def predict_tumor(image_array, model, labels_map):
    # O modelo espera um lote (batch), então transformamos (64,64,1) em (1,64,64,1)
    img_batch = np.expand_dims(image_array, axis=0)

    # Probabilidades (o seu 'probs')
    probs = model.predict(img_batch, verbose=0)

    # Pega o índice da maior probabilidade
    pred_idx = np.argmax(probs)

    # Mapeia o índice de volta para o nome (ex: 0 -> 'glioma')
    inv_map = {v: k for k, v in labels_map.items()}
    pred_label = inv_map[pred_idx]

    return pred_label, probs[0]

# Teste com a primeira imagem do conjunto de validação
for range
imagem_teste = X_val[0]
classe_real_idx = y_val[0]
classe_real_nome = [k for k, v in labels_map.items() if v == classe_real_idx][0]

predicao, probabilidades = predict_tumor(imagem_teste, model, labels_map)

print("-" * 30)
print(f"Classe Real: {classe_real_nome}")
print(f"Predição do Modelo: {predicao}")
print(f"Confiança: {probabilidades[np.argmax(probabilidades)]*100:.2f}%")

------------------------------
Classe Real: notumor
Predição do Modelo: notumor
Confiança: 27.73%
